## Concrete Strength

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.metrics import r2_score
import os
os.chdir(r"C:\Training\Academy\Statistics (Python)\Cases\Concrete Strength")

In [8]:
concrete = pd.read_csv("Concrete_Data.csv")
X = concrete.drop('Strength', axis=1)
y = concrete['Strength']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=24)

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred)
print(r2_lr)

0.5771752777048792


In [18]:
el = ElasticNet()
el.fit(X_train, y_train)
y_pred = el.predict(X_test)
r2_el = r2_score(y_test, y_pred)
print(r2_el)

0.5766806310401154


In [20]:
dtr = DecisionTreeRegressor(random_state=24)
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
r2_dtr = r2_score(y_test, y_pred)
print(r2_dtr)

0.83089474226832


In [22]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtr)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.7641542071921111

**Weighted Average**

In [27]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtr)],
                        weights=[0.15, 0.15, 0.7])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.851627386265932

OR

In [32]:
voting = VotingRegressor([('LR',lr),('EL',el),('DT',dtr)],
                        weights=[r2_lr, r2_el, r2_dtr])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.7955821089551822

Voting using `estimators` of same algorithm

In [39]:
dtr1 = DecisionTreeRegressor(random_state=24, max_depth=None)
dtr2 = DecisionTreeRegressor(random_state=24, max_depth=3)
dtr3 = DecisionTreeRegressor(random_state=24, max_depth=4)
voting = VotingRegressor([('M1',dtr1),('M2',dtr2),('M3',dtr3)],
                        weights=[0.7, 0.15, 0.15])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.8436797107172187

## Medical Cost Expenses

In [43]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector

In [49]:
os.chdir(r"C:\Training\Academy\Statistics (Python)\Cases\Medical Cost Personal")
med = pd.read_csv("insurance.csv")
X = med.drop('charges', axis=1)
y = med['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=24)

In [57]:
ohe = OneHotEncoder(handle_unknown='ignore')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False)
dtr = DecisionTreeRegressor(random_state=24)
pipe_dtr = Pipeline([('CT',ct),('DT',dtr)])
lr = LinearRegression()
pipe_lr = Pipeline([('CT',ct),('LR',lr)])
el = ElasticNet()
pipe_el = Pipeline([('CT',ct),('EL',el)])

In [59]:
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print(r2_score(y_test, y_pred))

0.7665391799816872


In [61]:
pipe_el.fit(X_train, y_train)
y_pred = pipe_el.predict(X_test)
print(r2_score(y_test, y_pred))

0.49673651151300835


In [63]:
pipe_dtr.fit(X_train, y_train)
y_pred = pipe_dtr.predict(X_test)
print(r2_score(y_test, y_pred))

0.7073555236369116


In [67]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DT',pipe_dtr)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(r2_score(y_test, y_pred))

0.7702125264403765


Weighted Average

In [72]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DT',pipe_dtr)],
                         weights=[4, 2, 4])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(r2_score(y_test, y_pred))

0.79295229109966
